# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/diego/Downloads/Starter_Code-7/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,289.14,94,8,0.45,PT,1712892814
1,1,katabu,-4.9333,122.5167,302.21,77,33,1.79,ID,1712893020
2,2,ta`u,-14.2336,-169.5144,302.38,76,40,4.97,AS,1712893021
3,3,savanna-la-mar,18.2190,-78.1332,300.12,88,83,1.42,JM,1712893021
4,4,carauari,-4.8828,-66.8958,296.74,99,94,0.86,BR,1712893022


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# %%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= "OSM",
    color= 'City',

)

# Display the map
map_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values

weather_con_df = city_data_df.loc[(city_data_df['Max Temp'] < 85) & (city_data_df['Max Temp'] > 65)]
weather_con_df = city_data_df.loc[city_data_df['Wind Speed'] < 10]
weather_con_df = city_data_df.loc[city_data_df['Cloudiness'] < 5]


# Drop any rows with null values
weather_con_df.dropna()

# Display sample data
weather_con_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,kerikeri,-35.2268,173.9474,296.99,71,0,3.34,NZ,1712893024
10,10,tura,25.5198,90.2201,307.86,21,3,1.82,IN,1712893025
13,13,chengde,40.9725,117.9361,299.06,31,3,2.97,CN,1712893026
19,19,adrar,20.5022,-10.0711,300.78,14,0,7.81,MR,1712893030
21,21,ancud,-41.8697,-73.8203,281.63,70,0,2.84,CL,1712892829
...,...,...,...,...,...,...,...,...,...,...
532,532,plastun,44.7561,136.2939,288.24,49,2,3.40,RU,1712893279
534,534,pierre,44.3683,-100.3510,280.53,47,0,3.60,US,1712893253
543,543,port shepstone,-30.7414,30.4550,288.75,57,0,6.90,ZA,1712893285
544,544,puerto la cruz,10.2167,-64.6167,298.99,84,0,4.45,VE,1712893285


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_con_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,kerikeri,NZ,-35.2268,173.9474,71,
10,tura,IN,25.5198,90.2201,21,
13,chengde,CN,40.9725,117.9361,31,
19,adrar,MR,20.5022,-10.0711,14,
21,ancud,CL,-41.8697,-73.8203,70,
...,...,...,...,...,...,...
532,plastun,RU,44.7561,136.2939,49,
534,pierre,US,44.3683,-100.3510,47,
543,port shepstone,ZA,-30.7414,30.4550,57,
544,puerto la cruz,VE,10.2167,-64.6167,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
          'apiKey': geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon}, {lat}, {radius}'
    params["bias"] = f'proximity:{lon}, {lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kerikeri - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
chengde - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
dawson creek - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
la ronge - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
el granada - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
narrabri - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
queenstown - nearest hotel: No hotel found
mingshui - nearest hotel: No hotel found
madhubani - nearest hotel: No hotel found
karratha - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
9,kerikeri,NZ,-35.2268,173.9474,71,No hotel found
10,tura,IN,25.5198,90.2201,21,No hotel found
13,chengde,CN,40.9725,117.9361,31,No hotel found
19,adrar,MR,20.5022,-10.0711,14,No hotel found
21,ancud,CL,-41.8697,-73.8203,70,No hotel found
...,...,...,...,...,...,...
532,plastun,RU,44.7561,136.2939,49,No hotel found
534,pierre,US,44.3683,-100.3510,47,No hotel found
543,port shepstone,ZA,-30.7414,30.4550,57,No hotel found
544,puerto la cruz,VE,10.2167,-64.6167,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
#%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points('Lng',
                                   'Lat',
                                   color = 'City',
                                   geo= True,
                                   tiles= 'OSM'
                                   )

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)